In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import os

In [3]:
os.chdir('..')
trained_model_path = Path('artifacts/model_training/model.h5')
model = tf.keras.models.load_model(trained_model_path)

In [4]:
convertor = tf.lite.TFLiteConverter.from_keras_model(model)


In [5]:
tflite_model = convertor.convert()

INFO:tensorflow:Assets written to: C:\Users\DJ\AppData\Local\Temp\tmpc5zbaifu\assets


INFO:tensorflow:Assets written to: C:\Users\DJ\AppData\Local\Temp\tmpc5zbaifu\assets


In [10]:
tflite_model_dir = Path('artifacts/model_training')

In [11]:
tflite_model_filepath = tflite_model_dir/'VGG16_model.tflite'

In [13]:
tflite_model_filepath.write_bytes(tflite_model)

59068424

In [14]:
convertor.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_quant_model = convertor.convert()

tflite_quant_model_filepath = tflite_model_dir/'VGG16_quant_model.tflite'
tflite_quant_model_filepath.write_bytes(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\DJ\AppData\Local\Temp\tmpfua8z7dw\assets


INFO:tensorflow:Assets written to: C:\Users\DJ\AppData\Local\Temp\tmpfua8z7dw\assets


14837584

WindowsPath('artifacts/model_training/VGG16_model.tflite')

# Loading TFLITE model

In [20]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_quant_model_filepath))
interpreter.allocate_tensors()

In [19]:
import matplotlib.pyplot as plt
input_image = plt.imread('artifacts/data_ingestion/PetImages/Cat/0.jpg')
input_image.shape

(375, 500, 3)

In [21]:
interpreter.get_input_details()

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([  1, 224, 224,   3]),
  'shape_signature': array([ -1, 224, 224,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [23]:
resized_img = tf.image.resize(input_image, (224,224))

In [24]:
resized_img = tf.expand_dims(resized_img, 0)

In [25]:
resized_img.shape

TensorShape([1, 224, 224, 3])

In [28]:
input_idx = interpreter.get_input_details()[0]['index']
output_idx = interpreter.get_output_details()[0]['index']

In [29]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 50,
  'shape': array([1, 2]),
  'shape_signature': array([-1,  2]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [30]:
interpreter.set_tensor(input_idx, resized_img)
interpreter.invoke()
pred = interpreter.get_tensor(output_idx)

In [31]:
pred

array([[1., 0.]], dtype=float32)

In [37]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_quant_model_filepath))
interpreter.allocate_tensors()
interpreter.set_tensor(input_idx, input_image)
interpreter.invoke()
pred = interpreter.get_tensor(output_idx)
pred

ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_input_1:0 

In [36]:
input_image = input_image/255.

In [38]:
tf.float32(input_image)

TypeError: 'DType' object is not callable